<a href="https://colab.research.google.com/github/omidshm/tradingview-sentiment/blob/cooking-jupyter-mvp/tradingview_sentiment_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install requests-HTML

In [5]:
import requests
from requests_html import HTMLSession

In [6]:
import json

In [23]:
import pandas as pd

In [7]:
url = 'https://www.tradingview.com/markets/cryptocurrencies/ideas/'

try:
    session = HTMLSession()
    response = session.get(url)
     
except requests.exceptions.RequestException as e:
    print(e)

In [9]:
widget_xpath = '//*[@id="js-category-content"]/div/div/div/div/div/div/div[2]/div/div'
tv_widgets = response.html.xpath(widget_xpath)

In [29]:
idea_item_sample = {
    'symbol':'BTCUSDT',
    'default_sentiment':'LONG',
    'timeframe':'60',
    'title':'BTC LOOKS BULLISH',
    'link':'https://tgview.com/sdsfsdfsfsdf',
    'timestamp':'date'
}

idea_items = []

In [30]:
# get page items and return idea_items
def get_item(widget):
  idea_item = {}

  symbol_xpath = '//div[2]/div/a'
  sentiment_xpath = '//span[3]/span[2]'
  timeframe_xpath = '//div[2]/span[2]'
  title_xpath = '//div[1]/a'
  link_xpath = '//div[1]/a'
  date_path = '//div[4]/span[3]/span/span'
  BASE_TGVIEW_URL = 'https://www.tradingview.com'

  try:
    idea_item['symbol'] = widget.xpath(symbol_xpath)[0].text
    
  except:
    print('Logging: some adds blocked')
  else:
    try:
      idea_item['default_sentiment'] = widget.xpath(sentiment_xpath)[0].text
    except:
      idea_item['default_sentiment'] = ''
    
    idea_item['timeframe'] = widget.xpath(timeframe_xpath)[0].text
    idea_item['title'] = widget.xpath(title_xpath)[0].text
    idea_item['timestamp'] = widget.xpath(date_path)[0].attrs['data-timestamp']
    
    idea_item['link'] = BASE_TGVIEW_URL+widget.xpath(link_xpath)[0].attrs['href']

    idea_items.append(idea_item)

    print(f'{idea_item} added to items list')

  
  

In [31]:
for widget in tv_widgets:
  get_item(widget)

{'symbol': 'BTCUSDT', 'default_sentiment': '', 'timeframe': '1D', 'title': '💡 BTC/USDT', 'timestamp': '1645019529.0', 'link': 'https://www.tradingview.com/chart/BTCUSDT/AfPOhkxw-BTC-USDT/'} added to items list
{'symbol': 'BTCUSDT', 'default_sentiment': '', 'timeframe': '60', 'title': 'BITCOIN - Update', 'timestamp': '1645025200.0', 'link': 'https://www.tradingview.com/chart/BTCUSDT/6wEWLyO4-BITCOIN-Update/'} added to items list
{'symbol': 'SHIBUSDT', 'default_sentiment': 'Long', 'timeframe': '240', 'title': 'SHIBA can go to 4000 sat this week and give around 30% profit', 'timestamp': '1645008095.0', 'link': 'https://www.tradingview.com/chart/SHIBUSDT/TgNfRwRY-SHIBA-can-go-to-4000-sat-this-week-and-give-around-30-profit/'} added to items list
Logging: some adds blocked
{'symbol': 'BTCUSDT', 'default_sentiment': 'Long', 'timeframe': '60', 'title': 'BITCOIN IS STILL BULLISH AND CAN GO TO 45.000-47.000 VERY SOON', 'timestamp': '1645007292.0', 'link': 'https://www.tradingview.com/chart/BTCU

In [32]:
items_df = pd.DataFrame(idea_items)

In [33]:
items_df

,symbol,default_sentiment,timeframe,title,timestamp,link
0,BTCUSDT,,1D,💡 BTC/USDT,1645019529.0,https://www.tradingview.com/chart/BTCUSDT/AfPO...
1,BTCUSDT,,60,BITCOIN - Update,1645025200.0,https://www.tradingview.com/chart/BTCUSDT/6wEW...
2,SHIBUSDT,Long,240,SHIBA can go to 4000 sat this week and give ar...,1645008095.0,https://www.tradingview.com/chart/SHIBUSDT/TgN...
3,BTCUSDT,Long,60,BITCOIN IS STILL BULLISH AND CAN GO TO 45.000-...,1645007292.0,https://www.tradingview.com/chart/BTCUSDT/NAz5...
4,BTCUSD,,1D,BTC Bitcoin: Predicting The Future,1645046536.0,https://www.tradingview.com/chart/BTCUSD/3Zvud...
5,DOGEUSDT,Long,720,"DOGE near resistance line, ready to break it a...",1645008863.0,https://www.tradingview.com/chart/DOGEUSDT/gdC...
6,BTCUSD,,1D,BTCUSD next possible paths here,1644996593.0,https://www.tradingview.com/chart/BTCUSD/zdv85...
7,ETHUSD,Long,480,ETHUSD Will Move Higher! Buy!,1644998236.0,https://www.tradingview.com/chart/ETHUSD/evZ7g...
8,BTCUSD,,30,How to Trade & Sell Options?,1645031720.0,https://www.tradingview.com/chart/BTCUSD/zIONk...
9,LUNAUSD,Long,60,LUNAUSD H1: 50% gains XABCD BEST LEVEL TO BUY(...,1644997703.0,https://www.tradingview.com/chart/LUNAUSD/fQrA...
